In [1]:
from sklearn.datasets import fetch_california_housing
import pandas as pd

# Load California Housing Dataset
california = fetch_california_housing(as_frame=True)
df = pd.DataFrame(california.data, columns=california.feature_names)
df['MedHouseValue'] = california.target  # Target variable
df

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseValue
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422
...,...,...,...,...,...,...,...,...,...
20635,1.5603,25.0,5.045455,1.133333,845.0,2.560606,39.48,-121.09,0.781
20636,2.5568,18.0,6.114035,1.315789,356.0,3.122807,39.49,-121.21,0.771
20637,1.7000,17.0,5.205543,1.120092,1007.0,2.325635,39.43,-121.22,0.923
20638,1.8672,18.0,5.329513,1.171920,741.0,2.123209,39.43,-121.32,0.847


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

x, y = df.drop('MedHouseValue', axis=1), df['MedHouseValue']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test_transformed = scaler.transform(x_test)

len(x_train), len(x_test)

(16512, 4128)

In [3]:
import mikegrad as mg

x_train = mg.from_numpy(x_train)
y_train = mg.from_pandas(y_train)
x_test = mg.from_pandas(x_test)
y_test = mg.from_pandas(y_test)
x_test_transformed = mg.from_numpy(x_test_transformed)

len(x_train), len(x_test), len(x_train[0])

(16512, 4128, 8)

In [4]:
from mikegrad.nn import MLP
from mikegrad.losses import MSELoss
from mikegrad.optim import Adam
from mikegrad.utils import train

model = MLP(nin=8,
            nouts=[16, 8, 1],
            act= mg.nn.ReLU())

loss_fn = MSELoss()
lr=1e-2
optimizer = Adam(model.parameters(), lr=lr)
nepochs=5
batch_size=64

In [5]:
train(model, x_train, y_train, loss_fn, optimizer, nepochs, batch_size)

Epochs:   0%|          | 0/5 [00:00<?, ?it/s]

Epochs:  20%|██        | 1/5 [02:19<09:17, 139.49s/it, loss=1.699928]
























































































































































































































































































































































































































































































































Epochs:  60%|██████    | 3/5 [07:04<04:44, 142.00s/it, loss=0.617550]






































































































































































































































































































































































In [8]:
from mikegrad.utils import test

mse, y_pred = test(model, x_test_transformed, y_test, loss_fn)
mse

Testing: 100%|██████████| 4128/4128 [00:44<00:00, 93.43it/s, loss=0.034092]  


0.33898985224122785

In [7]:
import random

for i in range(10):
    random_idx = random.randint(0, len(x_test))
    print(f"Predicted: {y_pred[random_idx].data:.2f} | True: {y_test[random_idx].data:.2f}")

Predicted: 1.46 | True: 0.94
Predicted: 0.96 | True: 0.64
Predicted: 1.93 | True: 2.42
Predicted: 1.21 | True: 0.81
Predicted: 2.00 | True: 2.44
Predicted: 2.03 | True: 1.91
Predicted: 4.98 | True: 5.00
Predicted: 1.20 | True: 1.69
Predicted: 2.49 | True: 3.15
Predicted: 2.10 | True: 1.71
